# Tokenomics and Simulation Report: Antigravity

## Introduction
This report presents the simulation of Dark tokenomics and its various components, including the Journey Phase Manager, Treasury, Jackpot, and Pool modules. The simulation tracks the evolution of the Dark token price, NFT minting, and yield distribution across 33 journeys.

The objective of this report is to provide a comprehensive overview of the system's behavior and key metrics through detailed graphs and data accumulation.

## Table of Contents

* [Need for project like Antigravity?](#need-for-project-like-antigravity)

* [Introduction of Antigravity](#introduction-of-antigravity)

* [Goal of Antigravity and Impact Antigravity will make](#goal-of-antigravity-and-impact-antigravity-will-make)

* [What is $DARK?](#what-is-dark)

* [Components of Antigravity](#components-of-antigravity)

* [How FuelCells work?](#how-fuelcells-work)

* [How is yield generated](#how-is-yield-generated)

* [How Jackpot works?](#how-jackpot-works)

* [How will Evil Address operate?](#how-will-evil-address-operate)

* [Benefits of Evil Address to holders of $DARK and FuelCells?](#benefits-of-evil-address-to-holders-of-dark-and-fuelcells)

* [How is supply crisis created](#how-is-supply-crisis-created)

* [Tokenomics Simulations](#tokenomics-simulations)
    
    - [Simulation Variables](#variables)
    
    - [Token Distribution over journey](#token-distribution)
    
    - [Price of Dark Token over journeys](#price-of-dark-tokens-over-journey)
    
    - [Total Number of NFTs minted over journey](#total-number-of-nfts-minted-over-journeys)
    
    - [Total Number of NFTs minted in each Journey](#total-number-of-nfts-minted-in-each-journey)
    
    - [Cummulative Yield per NFT minted in Journey 1](#cummulative-yield-per-nft-minted-in-journey-1-in-dark-tokens)
    
    - [Cummulative Yield per NFT minted in Journey 1 in USD](#cummulative-yield-per-nft-minted-in-journey-1-in-usd)
    
    - [Expected ROI % for user minting in Journey 1](#return-on-investment)
    
    - [Cummulative yield given to NFTs till Journey 50](#cummulative-yield-given-to-nfts-till-journey-50)
    
    - [Change of Balance of Evil Address over journeys](#change-of-balance-of-evil-address-over-journeys)
    
    - [Expected Liquidity Pool trades](#expected-liquidity-pool-trades)

## Need for project like Antigravity?
Crypto needs a project that pushes the innovation of NFTs. This us an experiment in capturing and concentrating economic energy through tokenomics and defi.

## Introduction of Antigravity
Antigravity is the most dynamic story and set of tokenomics folded into a single project.

## Goal of Antigravity and Impact Antigravity will make
The goal of agproject.io is to create the best community in crypto hands down. We hope to break the 4 year bull and bear cycle.

## What is $DARK?
'$Dark' is a PRC20 token with a total supply of 1M tokens, no inflation and programmed scarcity. It is also the only token used to mint Fuel Cell NFTs.

## Components of Antigravity
The 3 components of Antigravity are it's story and art inspired by "Rings of Dust". The $Dark Matter Token, and the Fuel Cell NFT.

## How FuelCells work?
The Fuel Cell NFT is minted on a 1 to 1 ratio of the $Dark Matter token and has a limit of 1M. Users have 33 chances to mint the Fuel Cell NFTs and a 15 day window to mint each series. The Fuel Cell NFT also has 3 features: the Art from "Rings of Dust", the treasury yield paid every 105 days, and the lottery giveaways that take place in the 90 days following the minting phase.

## How is yield generated
Yield is generated from the minting of Fuel Cells. 77% of the $Dark Matter tokens that are used in minting fund the treasury. The treasury pays yield every 105 days into the Fuel Cell NFTs. The yeild can only be collected by breaking open the Fuel Cell NFT. Once the Fuel Cell NFT is broken open (unwrapped) it no longer receives yield from the treasury. Unbroken Fuel Cell NFTs receive yield from the treasury every 105 days.

## How Jackpot works?
20% of the $Dark Mater tokens used in minting go to funding the Jackpot address. When users mint a Fuel Cell NFT they're automatically entered into 3 drawings of the lottery that take place during the following 90 days.

## How will Evil Address operate?
The Evil Address has an unfair advantage in the Antgravity game. Not only does the Evil Adress receive the majority of $Dark Matter tokens (65%), but it also seeks to mint as many Fuel Cell NFTs as possible. By doing this the Evil Address hopes to hoard as many $Dark Matter tokens from winning lotteries and collecting yield from the treasury.

## Benefits of Evil Address to holders of $DARK and FuelCells?
The competitive nature of the Evil Address does remove the supply of $Dark Matter from the market. This may increase demand for the tokens.

## How is supply crisis created
Supply crises my arrise from the Evil Address hoarding tokens and NFTs, minting of the Fuel Cell NFTs, limited minting windows, limited minting time periods, fomo of getting into the next lotteries and an event called "The Rapture" where all unutilized $Dark Matter tokens will be destroyed and removed from supply following the final (33rd) minting phase

## Tokenomics Simulations

In [ ]:
# Import necessary packages and modules

# General packages
import numpy as np
import random
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from constants.constants import exports_dir

# Import custom modules
from modules.Account import Account
from modules.Dark import DarkToken
from modules.FuelCells import FuelCellsToken
from modules.JourneyPhaseManager import JourneyPhaseManager
from modules.Treasury import Treasury
from modules.LaunchControlCenter import LaunchControlCenter
from modules.EvilAddress import EvilAddress
from modules.Jackpot import Jackpot
from modules.Pool import Pool
from modules.Team import Team
from modules.DAIStableCoin import DAIStableCoin
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

# Set up initial configurations for the simulation
np.random.seed(42)
random.seed(42)


### Initialize accounts

In [ ]:
# Import necessary packages and modules

# Initialize Accounts

# Public account
public_account = Account()

# MagicBox account managed by the team
magic_box_account = Account()

# Dead account
dead_account = Account()

# Team account to collect share of Dark tokens used to mint NFTs
team_account = Team()

# Initialize Dark token
dark_token = DarkToken()

# Mint initial Dark token supply to Public, MagicBox, and Evil address
initial_dark_supply = 1000000  # Total initial supply
public_initial_balance = int(initial_dark_supply * 0.10)
magic_box_initial_balance = int(initial_dark_supply * 0.25)
evil_address_initial_balance = int(initial_dark_supply * 0.65)

dark_token.mint(public_account, public_initial_balance)
dark_token.mint(magic_box_account, magic_box_initial_balance)

# Initialize FuelCells token
fuel_cells_token = FuelCellsToken()

# Initialize JourneyPhaseManager
journey_phase_manager = JourneyPhaseManager()

# Initialize Treasury
treasury_account = Account()
treasury = Treasury(treasury_account, dark_token)

# Initialize Jackpot
jackpot_account = Account()
jackpot = Jackpot(jackpot_account, dark_token, journey_phase_manager, fuel_cells_token)

# Initialize LaunchControlCenter
lcc = LaunchControlCenter(dark_token, fuel_cells_token, treasury_account, jackpot_account, team_account, journey_phase_manager)

# Initialize EvilAddress
evil_address_account = EvilAddress(dark_token, fuel_cells_token, lcc, journey_phase_manager, evil_address_initial_balance)
dark_token.mint(evil_address_account, evil_address_initial_balance)

# Initialize Pool
dai_token = DAIStableCoin()
pool = Pool(dark_token, dai_token)

# Initialize accounts and variables summary
print("Initial Balances:")
print(f"Public Account: {dark_token.balance_of(public_account)} Dark tokens")
print(f"Treasury Contract Balance: {dark_token.balance_of(treasury_account)} Dark tokens")
print(f"Jackpot Contract Balance: {dark_token.balance_of(jackpot_account)} Dark tokens")
print(f"MagicBox Account: {dark_token.balance_of(magic_box_account)} Dark tokens")
print(f"Evil Address Contract Balance: {dark_token.balance_of(evil_address_account)} Dark tokens")
print(f"Team Account: {dark_token.balance_of(team_account)} Dark tokens")



### Data Storage and Helper Functions

In [ ]:
# Initialize data storage
prices = []
volumes = []
nfts_minted = [] # total NFTs minted
nfts_minted_per_journey = [] # NFTS minted in each journey
evil_address_balances = []
yields_per_journey = []
cumulative_yield_per_journey_1 = []
yields_per_journey_accumulated = []
yield_per_nft_per_journey = []
pool_dark_reserves = []
pool_dai_reserves = []
token_distribution = []

# Helper function to accumulate data
def accumulate_data(journey, phase, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account):
    prices.append({
        'journey': journey,
        'phase': phase,
        'price': pool.get_price()
    })
    volumes.append({
        'journey': journey,
        'phase': phase,
        'volume': pool.total_volume
    })
    evil_address_balances.append({
        'journey': journey,
        'phase': phase,
        'balance': dark_token.balance_of(evil_address_account)
    })
    pool_dark_reserves.append({
        'journey': journey,
        'phase': phase,
        'dark_reserve': pool.dark_reserve
    })
    pool_dai_reserves.append({
        'journey': journey,
        'phase': phase,
        'dai_reserve': pool.dai_reserve
    })
    token_distribution.append({
        'journey': journey,
        'phase': phase,
        'public_balance': dark_token.balance_of(public_account),
        'magic_box_balance': dark_token.balance_of(magic_box_account),
        'evil_address_balance': dark_token.balance_of(evil_address_account),
        'team_balance': dark_token.balance_of(team_account),
        'treasury_balance': dark_token.balance_of(treasury_account) - treasury.total_yield_distributed,
        'jackpot_balance': dark_token.balance_of(jackpot_account),
        'pool_balance': pool.dark_reserve,
        'yield_balance': treasury.total_yield_distributed
    })
    if phase == 3 and journey > 0:
        if journey <= 33:
            yield_for_journey = treasury.get_total_yield_for_journey(journey)
            yields_per_journey.append({
                'journey': journey,
                'total_yield': yield_for_journey
            })
            nfts_minted.append({
                "journey": journey,
                "nfts": fuel_cells_token.total_supply
            })
            supplyInPreviousJourney = 0
            if(nfts_minted_per_journey.__len__() != 0):
                supplyInPreviousJourney = nfts_minted[journey-2]["nfts"]

            nfts_minted_per_journey.append({
                "journey": journey,
                "nftsMinted": fuel_cells_token.total_supply - supplyInPreviousJourney
            })

        # Accumulate yield for NFTs minted in journey 1
        cumulative_yield_per_journey_1.append({
            'journey': journey,
            'total_yield': (treasury.get_total_yield_for_journey(1) / journey_phase_manager.get_nft_count(1))
        })

        yield_per_nft_per_journey.append({
            'journey': journey,
            'total_yield': treasury.get_total_yield_for_journey(1)
        })
        # Accumulate yield for NFTs minted in previous journeys
        total_yield_accumulated = treasury.total_yield_distributed
        yields_per_journey_accumulated.append({
            'journey': journey,
            'total_yield': total_yield_accumulated
        })



### Initialize Liquidity Pool

In [ ]:
# Initialize Liquidity Pool

# Mint initial DAI supply to MagicBox account
initial_dai_supply = 300000  # Total initial DAI supply for liquidity
dai_token.mint(magic_box_account, initial_dai_supply)

# Define liquidity amounts
dark_liquidity_amount = 100000  # Dark tokens from MagicBox balance
dai_liquidity_amount = initial_dai_supply   # DAI tokens

# Add liquidity to the pool from MagicBox
pool.add_liquidity(magic_box_account, dark_liquidity_amount, dai_liquidity_amount)

# Print pool status after adding liquidity
print(f"Liquidity Pool Initialized:")
print(pool)
print(f"Initial DARK token price: {pool.get_price()} per DAI token")


## Utility Functions

In [ ]:
# Utility functions

# Helper function to simulate trades
def simulate_trades(account, pool, total_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token):
    for _ in range(total_trades):
        trade_type = np.random.choice(["buy", "sell"], p=[buy_pressure, sell_pressure])
        if trade_type == "buy":
            dai_amount = np.random.randint(10, 100)  # Random DAI amount to buy Dark tokens
            if dai_token.balance_of(account) < dai_amount:
                dai_token.mint(account, dai_amount)  # Mint DAI tokens if account has insufficient balance
            if pool.dai_reserve >= dai_amount:
                pool.buy(account, dai_amount)
                dark_tokens_purchased = dai_amount / pool.get_price()
                if journey_phase_manager.get_current_journey() <=33:
                    nft_purchase_amount = int(dark_tokens_purchased * nft_purchase_share_from_public_buy)
                    if journey_phase_manager.get_current_phase() == 1 and dark_tokens_purchased >= nft_purchase_amount:
                        # print(f"Minting more FUEL_CELLS from ${account.address}. Amount: {nft_purchase_amount}")
                        lcc.mint_nft(account, nft_purchase_amount)
        else:
            dark_amount = np.random.randint(10, 100)  # Random Dark amount to sell for DAI
            if dark_token.balance_of(account) >= dark_amount and pool.dark_reserve >= dark_amount:
                pool.sell(account, dark_amount)


## Variables

In [ ]:
# Variables for controlling buy/sell pressures and trades

buy_pressure = 0.6  # 60% of the trades will be buys
sell_pressure = 0.4  # 40% of the trades will be sells
total_trades = 10000  # Total number of trades in the journey
nft_purchase_share_from_public_buy = 0.3  # 30% of the Dark tokens from buys will be used for minting NFTs
public_nft_mint_percentage = 0.10  # 10% of the public Dark token balance will be used to mint NFTs
public_re_mint_from_lottery_percentage = 0.10 # 10% of the lottery winnings will be used to mint NFTs again

# Initial pool status
initial_dark_reserve = pool.dark_reserve
initial_dai_reserve = pool.dai_reserve
initial_price = pool.get_price()

print(f"Initial Dark reserve: {initial_dark_reserve}")
print(f"Initial DAI reserve: {initial_dai_reserve}")
print(f"Initial Dark token price: {initial_price}")
print(f"Buy Probability: {buy_pressure}")
print(f"Sell Probability: {sell_pressure}")
print(f"Total Trades per journey: {total_trades}")
print(f"% of the Dark tokens from buys will be used for minting NFTs: {nft_purchase_share_from_public_buy}")
print(f"% of the public Dark token balance will be used to mint NFTs: {public_nft_mint_percentage}")
print(f"% of the lottery winnings will be used to mint NFTs again: {public_re_mint_from_lottery_percentage}")


## Run through the journeys

In [ ]:
# # Journey 1 Simulation

# # Simulate the minting phase
# public_initial_balance = dark_token.balance_of(public_account)
# public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

# evil_address_nft_mint_amount = int(evil_address_account.starting_balance * evil_address_account.b * (evil_address_account.r ** (journey_phase_manager.get_current_journey() - 1)) / 100)

# # Public mints NFTs
# lcc.mint_nft(public_account, public_nft_mint_amount)

# # Evil address mints NFTs
# evil_address_account.mint_fuel_cells()

# # Update phase to lottery phase
# journey_phase_manager.increment_phase()

# # Capture the jackpot balance before the lottery
# initial_jackpot_balance = dark_token.balance_of(jackpot.account)

# # Conduct the lottery
# lottery_result = jackpot.conduct_lottery()
# print(lottery_result)

# # Total lottery payouts
# total_lottery_payouts = sum(jackpot.calculate_payouts(journey_phase_manager.get_current_journey(), initial_jackpot_balance))
# print(f"Total lottery payouts: {total_lottery_payouts}")

# # Public sells all their lottery winnings
# lottery_winnings = jackpot.get_lottery_winnings(journey_phase_manager.get_current_journey(), public_account)
# if lottery_winnings > 0:
#     pool.sell(public_account, lottery_winnings)

# print(f"Total lottery winnings for public: {lottery_winnings}")

# # Simulate trades in Phase 1
# phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
# simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

# # Update phase to yield distribution
# journey_phase_manager.increment_phase()

# # Simulate trades in Phase 2
# phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
# simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

# # Update phase to next journey
# journey_phase_manager.increment_phase()

# # Simulate trades in Phase 3
# phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
# simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

# # Distribute yield to NFT holders
# treasury.distribute_yield(journey_phase_manager)

# # Calculate the total yield given by the treasury
# total_yield_given = treasury.total_yield_distributed
# print(f"Total yield given by the treasury in Journey 1: {total_yield_given}")

# # Calculate the yield per NFT for public and evil address
# public_nft_count = fuel_cells_token.balance_of(public_account)
# evil_nft_count = fuel_cells_token.balance_of(evil_address_account)
# total_nft_count = public_nft_count + evil_nft_count

# yield_per_nft = total_yield_given / total_nft_count if total_nft_count else 0
# print(f"Total yield per NFT: {yield_per_nft}")

# # Final pool status
# final_dark_reserve = pool.dark_reserve
# final_dai_reserve = pool.dai_reserve
# final_price = pool.get_price()

# print(f"Final Dark reserve: {final_dark_reserve}")
# print(f"Final DAI reserve: {final_dai_reserve}")
# print(f"Final Dark token price: {final_price}")

# # Print summary of Journey 1
# print("Journey 1 Summary:")
# print(f"Public Account: {fuel_cells_token.balance_of(public_account)} FuelCells NFTs")
# print(f"Evil Address Account: {fuel_cells_token.balance_of(evil_address_account)} FuelCells NFTs")
# print(f"Treasury Balance: {dark_token.balance_of(treasury_account)} Dark tokens")
# print(f"Jackpot Balance: {dark_token.balance_of(jackpot_account)} Dark tokens")
# print(f"Pool: {pool}")
# print(f"Total trades: {total_trades}")
# print(f"Buy pressure: {buy_pressure * 100}%")
# print(f"Sell pressure: {sell_pressure * 100}%")
# print(f"Total lottery winnings per winner: {lottery_winnings / total_trades if total_trades else 0}")
# print(f"Total volume in Pool: {pool.total_volume}")

# # Move to next journey
# journey_phase_manager.increment_phase()


### Simualte for 50 journeys

In [ ]:
# Simulate 33 Journeys with data accumulation

accumulate_data(0, 1, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
accumulate_data(0, 2, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
accumulate_data(0, 3, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

evilPercentages = []
totalNftsMintedValue = 0
nftsMintedInEachJourneyValue = 0
totalNFTsMinted = []
nftsMintedInEachJourney = []
treasuryBalance = []

for journey in range(1, 51):
    if journey<=33:
        print()
        print(f"Simulating Journey {journey}")

        # Phase 1: Minting Phase
        public_initial_balance = dark_token.balance_of(public_account)
        public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

        # Public mints NFTs
        lcc.mint_nft(public_account, public_nft_mint_amount)
        print(f"Public mints NFTs using: {public_nft_mint_amount} in journey {journey}, fuel cells balance: {fuel_cells_token.balance_of(public_account)}")

        # Evil address mints NFTs
        evil_address_account.mint_fuel_cells()

        # Simulate trades in Phase 1
        phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
        simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

        # Accumulate data at the end of Phase 1
        accumulate_data(journey, 1, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
        totalNftsMintedValue += fuel_cells_token.total_supply
        nftsMintedInEachJourneyValue = fuel_cells_token.total_supply - nftsMintedInEachJourneyValue
        totalNFTsMinted.append(totalNftsMintedValue)
        nftsMintedInEachJourney.append(nftsMintedInEachJourneyValue)
        # Increment to Phase 2
        journey_phase_manager.increment_phase()

        # Phase 2: Lottery Phase
        # Capture the jackpot balance before the lottery
        initial_jackpot_balance = dark_token.balance_of(jackpot.account)

        # Conduct the lottery
        lottery_result = jackpot.conduct_lottery()
        print(lottery_result)

        evil_address_jackpot_winning = jackpot.get_lottery_winnings(journey, evil_address_account)
        print(f"Evil Address lottery winning: {evil_address_jackpot_winning}")

        # Total lottery payouts
        total_lottery_payouts = sum(jackpot.calculate_payouts(journey_phase_manager.get_current_journey(), initial_jackpot_balance))
        # print(f"Total lottery payouts: {total_lottery_payouts}")

        # Public sells all their lottery winnings
        lottery_winnings = jackpot.get_lottery_winnings(journey_phase_manager.get_current_journey(), public_account)
        # 10% of the lottery winnings by public will be used to mint NFTs again in next journey
        tokens_to_be_sold = lottery_winnings * (1 - public_re_mint_from_lottery_percentage)
        if tokens_to_be_sold > 0:
            pool.sell(public_account, tokens_to_be_sold)

        print(f"Total lottery winnings for public: {lottery_winnings}")
        print(f"Total lottery winnings sold by public: {tokens_to_be_sold}")

        # Simulate trades in Phase 2
        phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
        simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        # print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

        # Accumulate data at the end of Phase 2
        accumulate_data(journey, 2, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

        # Increment to Phase 3
        journey_phase_manager.increment_phase()

        # transfer 3% to the team from recent evil address win
        team_tax_from_ea = evil_address_jackpot_winning * 0.03
        dark_token.transfer(evil_address_account, team_account, team_tax_from_ea)

        # transfer remaning winning to jackpot
        dark_token.transfer(evil_address_account, jackpot_account, evil_address_jackpot_winning - team_tax_from_ea)


        # Phase 3: Yield Distribution Phase
        # Simulate trades in Phase 3
        phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
        simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        # print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

        # Distribute yield to NFT holders
        treasury.distribute_yield(journey_phase_manager)

        # Accumulate data at the end of Phase 3
        accumulate_data(journey, 3, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

        # Increment to next journey (Phase 1 of the new journey)
        if journey == 33:
            dark_token.transfer(public_account, dead_account, dark_token.balance_of(public_account))
        journey_phase_manager.increment_phase()
    else:
        # Distribute yield to NFT holders
        treasury.distribute_yield(journey_phase_manager)
        # print(f"Simulating Journey {journey}")

        # Phase 1: Minting Phase
        public_initial_balance = dark_token.balance_of(public_account)
        public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

        # Simulate trades in Phase 1
        phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
        simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        # print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

        # Accumulate data at the end of Phase 1
        accumulate_data(journey, 1, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
        totalNftsMintedValue += fuel_cells_token.total_supply
        nftsMintedInEachJourneyValue = fuel_cells_token.total_supply - nftsMintedInEachJourneyValue
        totalNFTsMinted.append(totalNftsMintedValue)
        nftsMintedInEachJourney.append(nftsMintedInEachJourneyValue)
        # Increment to Phase 2
        journey_phase_manager.increment_phase()

        # Simulate trades in Phase 2
        phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
        simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        # print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

        # Accumulate data at the end of Phase 2
        accumulate_data(journey, 2, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

        # Increment to Phase 3
        journey_phase_manager.increment_phase()

        # Phase 3: Yield Distribution Phase
        # Simulate trades in Phase 3
        phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
        simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
        # print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

        # Distribute yield to NFT holders
        treasury.distribute_yield(journey_phase_manager)

        # Accumulate data at the end of Phase 3
        accumulate_data(journey, 3, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
        journey_phase_manager.increment_phase()
    treasuryBalance.append({
        'journey': journey,
        'balance': treasury.remaining_balance,
        'actualBalance': dark_token.balance_of(treasury_account)
    })

# totalNFTsMinted_df = pd.DataFrame(totalNFTsMinted)
# print(totalNFTsMinted_df)

# nftsMintedInEachJourney_df = pd.DataFrame(nftsMintedInEachJourney)
# print(nftsMintedInEachJourney_df)

### Graphs

In [ ]:
# Generate Graphs

# Convert accumulated data to DataFrames
price_df = pd.DataFrame(prices)
volume_df = pd.DataFrame(volumes)
nfts_minted_df = pd.DataFrame(nfts_minted)
nfts_minted_per_journey_df = pd.DataFrame(nfts_minted_per_journey)
evil_address_balance_df = pd.DataFrame(evil_address_balances)
yields_df = pd.DataFrame(yields_per_journey)
yields_accumulated_df = pd.DataFrame(yields_per_journey_accumulated)
cumulative_yield_df = pd.DataFrame(cumulative_yield_per_journey_1)
yield_per_nft_df = pd.DataFrame(yield_per_nft_per_journey)
pool_dark_reserve_df = pd.DataFrame(pool_dark_reserves)
pool_dai_reserve_df = pd.DataFrame(pool_dai_reserves)
token_distribution_df = pd.DataFrame(token_distribution)
evil_address_balance_df.to_csv("./EvilAddressbalanceChange.csv")


### Token Distribution

In [ ]:

# Plot Token distribution among the accounts across the journeys in a stacked area chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['magic_box_balance'], mode='lines', stackgroup='one', name='Magic Box Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['pool_balance'], mode='lines', stackgroup='one', name='Pool', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['treasury_balance'], mode='lines', stackgroup='one', name='Treasury Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['jackpot_balance'], mode='lines', stackgroup='one', name='Jackpot Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['evil_address_balance'], mode='lines', stackgroup='one', name='Evil Address Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['public_balance'], mode='lines', stackgroup='one', name='Public Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['team_balance'], mode='lines', stackgroup='one', name='Team Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['yield_balance'], mode='lines', stackgroup='one', name='Yield', hoverinfo='x+y'))
fig.update_layout(title='Token Distribution Among Accounts Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Token Balance')
fig.show()

### Price of Dark Tokens over Journey

In [ ]:
# Plot Price of Dark token across journeys and their phase
fig = go.Figure()
fig.add_trace(go.Scatter(x=price_df['journey'] + price_df['phase'] / 3, y=price_df['price'], mode='lines+markers', name='Price of Dark Token'))
fig.update_layout(title='Price of Dark Token Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Price of Dark Token')
fig.show()



### Total Number of NFTs minted over Journeys

In [ ]:

# Plot Total number of NFTs minted over journey
fig = go.Figure()
fig.add_trace(go.Bar(x=nfts_minted_df['journey'], y=nfts_minted_df['nfts'], name='Total NFTs Minted'))
fig.update_layout(title='Total Number of NFTs Minted over journey', xaxis_title='Journey', yaxis_title='Total NFTs Minted')
fig.show()


### Total Number of NFTs minted in each journey

In [ ]:

# Plot Number of NFTs minted in each journey
fig = go.Figure()
fig.add_trace(go.Bar(x=nfts_minted_per_journey_df['journey'], y=nfts_minted_per_journey_df['nftsMinted'], name='Total NFTs Minted'))
fig.update_layout(title='Total Number of NFTs Minted in Each Journey', xaxis_title='Journey', yaxis_title='Total NFTs Minted')
fig.show()


### Cummulative yield per NFT minted in Journey 1 in DARK tokens

In [ ]:
# Plot Cumulative yield per journey for NFTs minted in journey 1
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_yield_df['journey'], y=cumulative_yield_df['total_yield'], mode='lines+markers', name='Cumulative Yield for NFTs Minted in Journey 1'))
fig.update_layout(title='Cumulative Yield for NFTs Minted in Journey 1', xaxis_title='Journey', yaxis_title='Cumulative Yield (DARK)')
fig.show()

### Cummulative yield per NFT minted in Journey 1 in USD

In [ ]:
# Plot Cumulative yield per journey for NFTs minted in journey 1
# Filtering df1 where phase == 3
df1_phase3 = price_df[price_df['phase'] == 3]

# Merging df1_phase3 with df2 on 'journey'
merged_df = pd.merge(df1_phase3, cumulative_yield_df, on='journey')

# Creating the new DataFrame with the required calculation
result_df = pd.DataFrame({
    'journey': merged_df['journey'],
    'total_yield_usd': merged_df['price'] * merged_df['total_yield']
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=result_df['journey'], y=result_df['total_yield_usd'], mode='lines+markers', name='Cumulative Yield for NFTs Minted in Journey 1 in USD'))
fig.update_layout(title='Cumulative Yield for NFTs Minted in Journey 1', xaxis_title='Journey', yaxis_title='Cumulative Yield (USD)')
fig.show()

### Return on Investment

In [ ]:
# Plot Cumulative yield per journey for NFTs minted in journey 1
# Creating the new DataFrame with the required calculation
price_value = price_df[(price_df['journey'] == 0) & (price_df['phase'] == 1)]['price'].values[0]
roi_df = pd.DataFrame({
    'journey': result_df['journey'],
    'roi': result_df['total_yield_usd'] / price_value * 100
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=roi_df['journey'], y=roi_df['roi'], mode='lines+markers', name='Return on Investment in Journey 1'))
fig.update_layout(title='Cumulative Yield for NFTs Minted in Journey 1', xaxis_title='Journey', yaxis_title='ROI (%)')
fig.show()

### Cummulative yield given to NFTs till Journey 50

In [ ]:

# Plot Total yield generated by NFT minted in each journey
fig = go.Figure()
fig.add_trace(go.Bar(x=yields_accumulated_df['journey'], y=yields_accumulated_df['total_yield'], name='Total Yield Generated'))
fig.update_layout(title='Total Yield Generated by NFTs Minted in Each Journey', xaxis_title='Journey', yaxis_title='Total Yield')
fig.show()

### Change of Balance of Evil Address over journeys

In [ ]:


# Plot Change of balance of Evil address across each journey and their phase
fig = go.Figure()
fig.add_trace(go.Scatter(x=evil_address_balance_df['journey'] + evil_address_balance_df['phase'] / 3, y=evil_address_balance_df['balance'], mode='lines+markers', name='Evil Address Balance'))
fig.update_layout(title='Change of Balance of Evil Address Across Each Journey and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Balance of Evil Address')
fig.show()


### Expected Liquidity Pool trades

In [ ]:

# Plot Cumulative volume of trades across phases and journey
fig = go.Figure()
fig.add_trace(go.Scatter(x=volume_df['journey'] + volume_df['phase'] / 3, y=volume_df['volume'], mode='lines+markers', name='Cumulative Volume'))
fig.update_layout(title='Cumulative Volume of Trades Across Phases and Journey', xaxis_title='Journey and Phase', yaxis_title='Cumulative Volume')
fig.show()

# Plot Change in Dark token reserves in the pool
fig = go.Figure()
fig.add_trace(go.Scatter(x=pool_dark_reserve_df['journey'] + pool_dark_reserve_df['phase'] / 3, y=pool_dark_reserve_df['dark_reserve'], mode='lines+markers', name='Dark Reserve in Pool'))
fig.update_layout(title='Change in Dark Token Reserves in the Pool Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Dark Token Reserve')
fig.show()

# Plot Change in DAI token reserves in the pool
fig = go.Figure()
fig.add_trace(go.Scatter(x=pool_dai_reserve_df['journey'] + pool_dai_reserve_df['phase'] / 3, y=pool_dai_reserve_df['dai_reserve'], mode='lines+markers', name='DAI Reserve in Pool'))
fig.update_layout(title='Change in DAI Token Reserves in the Pool Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='DAI Token Reserve')
fig.show()